# ╰(*°▽°*)╯ Import stuff ╰(*°▽°*)╯

In [2]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
# import sklearn
# from sklearn import ensemble
# from sklearn.ensemble import RandomForestClassifier
import warnings
import gc


- import MetabolitesFromInChI.csv

In [7]:
df = pd.read_csv("https://raw.githubusercontent.com/meidelien/Metabolic-network-layout-using-biochemical-coordinates/main/Notebooks/MetabolitesFromInChIKey.csv")

# Creating new df with descriptors
- Iterating over InChI's from MetabolitesFromInChI df 
- Storing DescriptorCalculator output to descrs list
- Creating molDes df & appending Descriptors to each column


In [ ]:
mol_list = []

for i in df.InChI:
    mol = Chem.MolFromInchi(i)
    mol_list.append(mol)


desc_list=[i[0] for i in Descriptors._descList]
descriptor = MoleculeDescriptors.MolecularDescriptorCalculator(desc_list)

descrs =  []

for i in range(len(mol_list)):
    descrs.append(descriptor.CalcDescriptors(mol_list[i]))
molDes = pd.DataFrame(descrs, columns= desc_list)
molDes.head(5)

- Make csv cus reasons

In [22]:
molDes.to_csv("chemical_properties.csv", index = False)

In [3]:
data1 = pd.read_csv("MetabolitesFromInChIKeyWIP.csv")
data2 = pd.read_csv("chemical_properties.csv")


- Concatatenating MetabolitesFromInChIKey & chemical_properties.csv

In [4]:
result = pd.concat([data1, data2], axis=1).reindex(data1.index)

In [40]:
result.to_csv("Chemical_descriptors.csv", index = False)

In [5]:
df = pd.read_csv("Chemical_descriptors.csv")

In [6]:
Species_iD = ["M_h_c", "M_h_c", "M_nadh_c","M_acon_C_c","M_accoa_c","M_6pgl_c", "M_succoa_c", "M_glc__D_e","M_mal__L_c", "M_cit_c", "M_13dpg_c", "M_etoh_c","M_co2_c" ,"M_ru5p__D_c", "M_xu5p__D_c", "M_q8h2_c", "M_akg_c", "M_pyr_c", "M_q8_c", "M_succ_c", "M_e4p_c","M_ac_c","M_2pg_c", "M_actp_c","M_atp_c", "M_gln__L_c", "M_for_c", "M_adp_c","M_oaa_c","M_nadp_c", "M_glx_c", "M_coa_c", "M_3pg_c", "M_amp_c","M_nh4_c", "M_fru_e", "M_pep_c", "M_pi_c", "M_lac__D_c", "M_s7p_c", "M_6pgc_c", "M_etoh_c","M_o2_c", "M_o2_e", "M_acald_c", "M_fum_c", "M_nadph_c", "M_g3p_c", "M_glu__L_c", "M_acald_c", "M_dhap_c", "M_f6p_c", "M_icit_c", "M_pi_c","M_h2o_c"]

In [ ]:
BiGG = ["h", "h", "nadh","acon_C", "accoa", "6pgl", "succoa", "glc__D", "mal__L","cit", "13dpg", "etoh", "co2", "ru5p__D", "xu5p__D", "q8h2", "akg", "pyr", "q8", "succ", "e4p", "ac", "2pg","actp", "atp", "gln__L", "for", "adp", "oaa", "nadp", "glx", "coa"     ]